In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [13]:
file = './data_results/4WT_all_fx.xlsx'
def recup_variable(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='variable')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
def recup_parameter(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='parameter')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
def recup_set(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='set')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
data_variable = recup_variable(file)
data_parameter = recup_parameter(file)
data_set = recup_set(file)

In [42]:
liste_reservoir = data_set['r']['n']

In [65]:
data_parameter['demand'][data_parameter['demand']['r']=='r2']

,r,t,Value
24,r2,t1,44.83
25,r2,t2,18.00
26,r2,t8,45.00
27,r2,t9,51.67
28,r2,t13,15.17
29,r2,t14,103.83
30,r2,t15,34.83
31,r2,t16,43.83
32,r2,t17,54.83
33,r2,t18,51.17


,t,r,Value
0,t1,r2,44.83
1,t2,r2,18.00
2,t3,NaN,NaN
3,t4,NaN,NaN
4,t5,NaN,NaN
5,t6,NaN,NaN
6,t7,NaN,NaN
7,t8,r2,45.00
8,t9,r2,51.67
9,t10,NaN,NaN


In [63]:
data_parameter['demand']

,r,t,Value
0,r1,t1,9.83
1,r1,t2,5.00
2,r1,t3,3.67
3,r1,t4,6.50
4,r1,t5,5.67
...,...,...,...
80,r4,t20,3.00
81,r4,t21,2.00
82,r4,t22,3.00
83,r4,t23,3.00


In [89]:
data_set['l']

,n,n.1,Unnamed: 2
0,s,j1,NaN
1,j1,j2,NaN
2,j1,r1,NaN
3,j1,r4,NaN
4,j2,r2,NaN
5,j2,r3,NaN


['r3', 'j2', 'j1', 's']

In [115]:
pd.DataFrame([['s',0,0]],columns=data_parameter['height'].columns)

,n,Value,Unnamed: 2
0,s,0,0


In [116]:
pd.concat([pd.DataFrame([['s',0,0]],columns=data_parameter['height'].columns),data_parameter['height'][data_parameter['height']['n'].isin(path)]])

,n,Value,Unnamed: 2
0,s,0,0.0
0,j1,30,NaN
1,j2,30,NaN
4,r3,45,NaN


In [120]:
data_variable['Charge'][(data_variable['Charge']['n'].isin(path))&(data_variable['Charge']['t']=='t1')]

,n,t,Lowerbound,Value,Upperbound,Marginal
0,s,t1,0,57.635219,inf,0
24,j1,t1,0,57.187030,inf,0
48,j2,t1,0,56.130101,inf,0
120,r3,t1,0,55.795376,inf,0


In [131]:
n = 'r2'
def path_to_n(n,data_set):
    path = [n]
    while n !='s':
        n = data_set['l'][data_set['l']['n.1']==n]['n'].iloc[0]
        path.append(n)
    return path

fig = go.Figure()
path = path_to_n(n,data_set)
fig.add_trace(
    go.Scatter(
        x = path,
        y = pd.concat([pd.DataFrame([['s',0,0]],columns=data_parameter['height'].columns),data_parameter['height'][data_parameter['height']['n'].isin(path)]])['Value'],
        name = 'hauteur'
    )
)
for t in data_set['t']['dim1']:
    if t == 't1':
        fig.add_trace(
        go.Scatter(
            x = path,
            y = data_variable['Charge'][(data_variable['Charge']['n'].isin(path))&(data_variable['Charge']['t']=='t1')]['Value'],
            name = t,
            # visible='legendonly'
        )
        )
    else:
        fig.add_trace(
        go.Scatter(
            x = path,
            y = data_variable['Charge'][(data_variable['Charge']['n'].isin(path))&(data_variable['Charge']['t']=='t1')]['Value'],
            name = t,
            visible='legendonly'
        )
        )
fig.update_layout(
    hovermode='x',
    # width=400
)
fig.show()

In [87]:
liste_reservoir = data_set['r']['n']


for i,reservoir in enumerate(liste_reservoir):
    fig = go.Figure()
    demand = pd.merge(data_set['t'][['dim1']].rename(columns={'dim1':'t'}),data_parameter['demand'][data_parameter['demand']['r']==reservoir],on=['t'],how='left')
    demand['Value'] = demand['Value'].fillna(0)
    fig.add_trace(
        go.Scatter(
            x = demand['t'],
            y = -demand['Value'],
            stackgroup='one',
            name = 'Demande'
        )
    )

    fig.add_trace(
        go.Scatter(
            x = data_set['t']['dim1'],
            y = sum(data_variable['Qpipe'][(data_variable['Qpipe']['n.1']==reservoir)&(data_variable['Qpipe']['n']==n)]['Value'] for n in data_set['l'][data_set['l']['n.1']==reservoir]['n']),
            stackgroup='two',
            name = 'Approvisionnement'
        )
    )
    fig.add_trace(
        go.Scatter(
            x = data_set['t']['dim1'],
            y = data_variable['v'][data_variable['v']['n']==reservoir]['Value']-data_variable['v'][data_variable['v']['n']==reservoir]['Lowerbound'],
            name = 'Volume - Volume min'
        )
    )
    fig.update_layout(
        hovermode='x',
        legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
                ),
        title = reservoir
    )
    fig.show()

In [51]:
import os

In [53]:
'out.xlsx'.endswith('.xlsx')

True

In [54]:
[file for file in os.listdir('./data_results') if file.endswith('.xlsx')]

['4WT_fx_night.xlsx', 'out.xlsx', '~$4WT_fx_night.xlsx']

In [31]:
data_parameter['psi'][data_parameter['psi']['degree']==2]['Value']

1   -0.006409
Name: Value, dtype: float64

In [50]:
data_parameter['psi']
X = np.linspace(0,100,100)
Y = data_parameter['psi'][data_parameter['psi']['degree']==2]['Value'].iloc[0]*X**2 + data_parameter['psi'][data_parameter['psi']['degree']==0]['Value'].iloc[0]
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = X,
        y = Y,
        name = 'Courbe de gain de charge théorique'
    )
)
for c in data_variable['Qpompe']['c'].unique():
    for d in data_variable['Qpompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['Value'],
                y               =   data_variable['Gpompe']['Value'],
                marker_color    =   color_map[d],
                name            =   f"Débit {c},{d}",
                # hoverinfo       =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['t'],
                mode='markers'
            )
        )
fig.update_layout(
    height= 800,
    hovermode='x'
)
fig.update_xaxes(range=[0,data_variable['Qpompe']['Value'].max()*1.1])
fig.update_yaxes(range=[data_variable['Gpompe']['Value'].min()*0.9,data_variable['Gpompe']['Value'].max()*1.1])
fig.show()

In [13]:
Z = 2
fig = go.Figure()
for r in data_variable['v']['n'].unique():
    fig.add_trace(
        go.Bar(
            x   =  data_variable['v'][data_variable['v']['n']==r]['t'],
            y   =  data_variable['v'][data_variable['v']['n']==r]['Value'],
            name=   r
        )
    )
for c in data_variable['Ppompe']['c'].unique():
    for d in data_variable['Ppompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x   =   data_variable['Ppompe'][(data_variable['Ppompe']['c']==c)&(data_variable['Ppompe']['d']==d)]['t'],
                y   =   data_variable['Ppompe'][(data_variable['Ppompe']['c']==c)&(data_variable['Ppompe']['d']==d)]['Value'],
                yaxis="y2",
                name=   f"({c},{d})"
            )
        )
fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        hovermode='x',
        yaxis=dict(
            title="m3" ,
            # range=[0,df["Consommation (MW)"].max()*1.1]
            ),
        yaxis2=dict(title="kW",
        # range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
        anchor="free",
        overlaying="y",
        side="right",
        position=1
        ),
        title = f"Etat du réseau de distribution d'eau - Coût : {Z} €"
    )
fig.show()
# fig.write_html("result.html")

In [56]:
[file for file in os.listdir('./data_results') if file.endswith('.xlsx')][0]

'4WT_fx_night.xlsx'

In [64]:
data_variable['Charge'][data_variable['Charge']['n']=='s']['Value']

0     57.635219
1     54.299841
2     54.558968
3     54.949131
4     55.367206
5     55.807872
6     56.265675
7     56.385046
8     53.760826
9     60.929469
10    53.366659
11    58.832858
12    61.051633
13    60.923149
14    55.287573
15    60.908369
16    60.909363
17    63.073191
18    63.073191
19    63.073191
20    63.073191
21    58.391783
22    63.073191
23    59.646449
Name: Value, dtype: float64

In [23]:
color_map   =   {
    'p1'    :   'red',
    'p2'    :   'blue',
    'p3'    :   'green'
}
color_map_light   =   {
    'p1'    :   'lightcoral',
    'p2'    :   'lightblue',
    'p3'    :   'lightgreen'
}
fig = go.Figure()
for c in data_variable['Son']['c'].unique():
    for d in data_variable['Son']['d'].unique():
        fig.add_trace(
            go.Bar(
                x               =   data_variable['Son'][(data_variable['Son']['c']==c)&(data_variable['Son']['d']==d)]['t'],
                y               =   10*data_variable['Son'][(data_variable['Son']['c']==c)&(data_variable['Son']['d']==d)]['Value'],
                marker_color    =   color_map_light[d],
                name            =   f"Statut {c},{d}"
            )
        )
fig.update_layout(
    barmode="stack"
)
fig.add_trace(
    go.Scatter(
        x               =   data_variable['Gpompe']['t'],
        y               =   data_variable['Gpompe']['Value'],
        marker_color    =   'black',
        line_dash       =   'dash',
        name            =   f"Gain Charge {c},{d}"
    )
)
for c in data_variable['Qpompe']['c'].unique():
    for d in data_variable['Qpompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['t'],
                y               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['Value'],
                marker_color    =   color_map[d],
                name            =   f"Débit {c},{d}"
            )
        )
fig.update_layout(
    hovermode='x',
    title = f"Etat des pompes du réseau de distribution d'eau - Coût : {Z} € "
    )
fig.show()

In [ ]:
fig = go.Figure()
for n in data_variable['Charge']['n'].unique():
    fig.add_trace(
        go.Scatter(
            x = data_variable['Charge'][data_variable['Charge']['n']==n]['t'],
            y = data_variable['Charge'][data_variable['Charge']['n']==n]['Value'],
            name = n
        )
    )
fig.update_layout(
    legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
    hovermode='x',
    title = f"Charge dans le réseau de distribution d'eau - Coût : {Z} € ",
    yaxis_title = "Charge (m)"
)
fig.show()